# Create Index
Purpose is to create a base Azure AI Search index to host the content

## Required for this step
- Azure AI Search Service

## Important
- Ensure that Semantic Ranker is enabled in Azure AI Search service
- If you choose to use an Embeddings Model other than ADA 002, make sure to check the dimensions and update the schema.json file accordlingly


In [2]:
# # Import required libraries  
import os  
import requests
import json


In [17]:
#Load the configuration details for the Cognitive Search Service and Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault
config = json.load(open("config.json"))
    
# Azure AI Search Config
search_service_name = config["search_service_name"]
search_service_url = "https://{}.search.windows.net/".format(search_service_name)
search_admin_key = config["search_admin_key"]
index_name = config["search_index_name"]
index_schema_file = config["search_index_schema_file"]
search_api_version = config["search_api_version"]
search_headers = {  
    'Content-Type': 'application/json',  
    'api-key': search_admin_key  
}  

#Azure OpenAI
azure_openai_api_base = config["openai_api_base"]
azure_openai_api_key = config["openai_api_key"]
azure_openai_embeddings_model = config["openai_embedding_model"]

print ('Search Service Name:', search_service_name)
print ('Index Name:', index_name)
print ('Azure OpenAI Base URL:', azure_openai_api_base)
print ('Azure OpenAI Embeddings Model:', azure_openai_embeddings_model)



In [14]:
# Load the Index Scheam - Using REST for this to leverage most recent vector capabilities
# NOTE: If you choose to use an Embeddings Model other than ADA 002, make sure to check the dimensions and update the 
# schema.json file accordlingly

with open(index_schema_file, "r") as f_in:
    index_schema = json.loads(f_in.read())
    index_schema['name'] = index_name
    index_schema['vectorSearch']['vectorizers'][0]['azureOpenAIParameters']['resourceUri'] = azure_openai_api_base
    index_schema['vectorSearch']['vectorizers'][0]['azureOpenAIParameters']['deploymentId'] = azure_openai_embeddings_model
    index_schema['vectorSearch']['vectorizers'][0]['azureOpenAIParameters']['apiKey'] = api_key


In [16]:
# Making the POST request to create the index  
delete_url = f"{search_service_url}/indexes/{index_name}?api-version={search_api_version}"  
response = requests.delete(delete_url, headers=search_headers)  
if response.status_code == 204:  
    print(f"Index {index_name} deleted successfully.")  
    # print(json.dumps(response.json(), indent=2))  
else:  
    print("Error creating index:")  
    print(response.json())  

# The endpoint URL for creating the index  
create_index_url = f"{search_service_url}/indexes?api-version={search_api_version}"  
response = requests.post(create_index_url, headers=search_headers, json=index_schema)  
  
# Check the response  
if response.status_code == 201:  
    print(f"Index {index_name} created successfully.")  
    # print(json.dumps(response.json(), indent=2))  
else:  
    print(f"Error creating index {index_name} :")  
    print(response.json())  
  


Index financial-docs deleted successfully.
Index financial-docs created successfully.
